In [ ]:
import chainer
import cv2
import numpy as np
import os
import seaborn as sns
from chainer import serializers as S
from datetime import datetime as dt
from ipywidgets import interact, IntSlider
from matplotlib import pyplot as plt
%matplotlib inline

from danmaku_env import DanmakuEnv
from model import Model

In [ ]:
chainer.global_config.train = False
chainer.global_config.enable_backprop = False

env = DanmakuEnv(0)
obs_space = env.observation_space
action_space = env.action_space

model = Model(obs_space.shape[0], action_space.n)
S.load_npz('trained_model_example.npz', model)

state = env.reset()
collision = False
step_count = 0

images = [env.render()]

MAX_STEP = 1000

while (step_count < MAX_STEP) and (not collision):
    pout, _ = model.pi_and_v(state[np.newaxis])
    #     action = pout.sample().data[0]
    action = pout.most_probable.data[0]
    state, _, collision, _ = env.step(action)
    step_count += 1
    images.append(env.render())

len(images)

In [ ]:
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = 12, 8
image_idx = IntSlider(0, 0, len(images) - 1, 1)


@interact(idx=image_idx)
def show_images(idx):
    plt.imshow(images[idx])
    plt.grid(False)
    plt.title('step {0:03d}'.format(idx))

In [ ]:
out_dir = 'test_result_' + dt.now().strftime('%Y%m%d%H%M%S')
os.makedirs(out_dir)

In [ ]:
for idx, image in enumerate(images):
    cv2.imwrite(
        os.path.join(out_dir, 'test_step_{0:06d}.png'.format(idx)),
        image[:, :, ::-1])